# Solve Pushing

In [1]:
import sys
sys.path.append('../rai-python/build')
import numpy as np
from robotic import ry
# import libry as ry
import time

[rai] util.cpp:initCmdLine:559(1) ** cmd line arguments: 'rai-pybind -python '
[rai] util.cpp:initCmdLine:562(1) ** run path: '/home/basti/Dokumente/Bachelorarbeit/rai-pcg/notebooks'
[rai] util.cpp:initCmdLine:563(1) ** rai path: '/root/local/rai'
[rai] graph.cpp:initParameters:1364(1) ** parsed parameters:
{python}



## Setting up a basic Config

In [3]:
C = ry.Config()
C.clear()

run = 1
file = 12
C.addFile(f'../agents/maze/pushing_turtle_{run}_log/generated/{file}.g')

C.view()

0

In [ ]:
# GET GOALS
goalNames = [s for s in frameNames if 'goal' in s]
print(goalNames)

goal = goalNames[0]
agent = 'robo'

## Path Optimization

In [ ]:
# X0 = C.getFrameState()

# komo = C.komo_path(1., 20, 10., True)

# komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
# komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);

# # robo inside goal
# komo.addObjective(times=[1.], feature=ry.FS.aboveBox, frames=[agent, goal], type=ry.OT.eq, scale=[1e2])

# komo.addObjective(times=[1.], feature=ry.FS.qItself, type=ry.OT.eq, order=1);


# # komo.addObjective(times=[1.], feature=ry.FS.position, frames=[roboFrame], type=ry.OT.eq, scale=[1, 1, 0], target=[1, 1, 0]);

# komo.optimize()
# komo.getReport()
# komo.view(False, "my solution")
# komo.view_play(False, 0.1)

## Create skeleton

In [ ]:
S = ry.Skeleton()

# touch goal
S.addEntry([1.], ry.SY.positionEq, [agent, goal])


## Test puzzel
# S.addEntry([1.], ry.SY.touch, [agent, 'obj'])
# S.addEntry([1., 2.], ry.SY.stable, [agent, 'obj'])
# S.addEntry([2.], ry.SY.above, ['obj', 'goal'])
# S.addEntry([2.,3.], ry.SY.stableOn, ['goal', 'obj'])


# # move box4-4 to goal 5-5
# S.addEntry([1.], ry.SY.touch, ['armL', 'box4-4'])
# S.addEntry([1., 2.], ry.SY.stable, ['armL', 'box4-4'])
# S.addEntry([2.], ry.SY.above, ['box4-4', 'goal5-5'])
# S.addEntry([2.,7.], ry.SY.stableOn, ['goal5-5', 'box4-4'])

# # move box2-4 to goal1-1
# S.addEntry([3.], ry.SY.touch, ['armL', 'box2-4'])
# S.addEntry([3., 4.], ry.SY.stable, ['armL', 'box2-4'])
# S.addEntry([4.], ry.SY.above, ['box2-4', 'goal1-1'])
# S.addEntry([4.,7.], ry.SY.stableOn, ['goal1-1', 'box2-4'])

# # move box2-2 to goal5-1
# S.addEntry([5.], ry.SY.touch, ['armL', 'box2-2'])
# S.addEntry([5., 6.], ry.SY.stable, ['armL', 'box2-2'])
# S.addEntry([6.], ry.SY.above, ['box2-2', 'goal5-1'])
# S.addEntry([6.,7.], ry.SY.stableOn, ['goal5-1', 'box2-2'])


## solve for waypoints: create a komo instance, create nlp instance, then call generic solver

In [ ]:
komo = S.getKomo_waypoints(C, 1e-1, 1e-2)
nlp = komo.nlp()
sol = ry.NLP_Solver()
sol.setProblem(nlp)
sol.setOptions( stopTolerance=1e-2 )
ret = sol.solve()
waypoints = komo.getPath_qAll()
# report on result, view, and play
print(ret)
#print(nlpW.report(2))
komo.view(True, 'waypoints solution')
komo.view_play(True, .2)
# store result

## solve for paths using RRT: for each phase create start-end problems, run RRT

In [ ]:
m = len(waypoints)
rrt_dofs = []
rrt_paths = []
for t in range(0,int(m)):
    # grab config and waypoints
    [Ctmp, q0, q1] = S.getTwoWaypointProblem(t, komo)
    Ctmp.setJointState(q0);
    Ctmp.view(True, 'waypoint configuration phase ' + str(t) + ' START')
    Ctmp.setJointState(q1);
    Ctmp.view(True, 'waypoint configuration phase ' + str(t) + ' STOP')

    # call path finder
    sol = ry.PathFinder()
    sol.setProblem(Ctmp, q0, q1)
    ret = sol.solve()
    rrt_paths.append(ret.x)
    rrt_dofs.append(Ctmp.getDofIDs())

    #display the rrt path
    for i in range(0,ret.x.shape[0]):
        Ctmp.setJointState(ret.x[i])
        Ctmp.view(False, 'rrt path ' + str(i))
        time.sleep(.05)

## solve for full path: create a komo instance, initialize with waypoints & rrt paths, solve

In [ ]:
komo = S.getKomo_path(C, 20, .2, -1, 1e-2)
komo.initWithWaypoints(waypoints)
komo.view(True, 'init with waypoints only')
for t in range(0,int(m)):
    komo.initPhaseWithDofsPath(t, rrt_dofs[t], rrt_paths[t], True)
    komo.view(True, 'init with RRT phase ' + str(t))
nlp = komo.nlp()
sol = ry.NLP_Solver()
sol.setProblem(nlp)
sol.setOptions( stopTolerance=1e-2 )
ret = sol.solve()
# report on result, view, and play
print(ret)
#print(nlp.report(2))
komo.view(True, 'path solution')
komo.view_play(True, .2)